## Settings

In [1]:
# 一つ上の階層からモジュールを参照できるようにする
import sys
sys.path.append('..')

In [2]:
# ロギング用
import os
from datetime import datetime, timezone, timedelta
from logs.logger import create_logger, get_logger
# モデル作成補助用
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from features.funcs import load_feather
from evals.funcs import get_pred_result, get_acc_and_logloss, print_conf_matrix
from inputs.funcs import load_data
# モデル作成用
from models.keras import ModelKeras
from keras.layers import Dense, Dropout

In [3]:
# モジュールの変更を自動的に反映する
%load_ext autoreload
%autoreload 2

In [4]:
# loggingの設定を行う
today = datetime.now(timezone(timedelta(hours=9)))
exp_version = today.strftime('%Y%m%d')
os.environ['exp_version'] = exp_version
create_logger(exp_version)

In [5]:
os.listdir('../features/nn')

['age_train_std_te.feather',
 'age_test_std_mean.feather',
 'age_train_std_mean.feather',
 'age_test_std_te.feather']

## Keras

### Base

In [6]:
train_x, train_y, test_x = load_data()

In [7]:
train_x = train_x.drop(['Name', 'Cabin', 'Ticket', 'Sex', 'Embarked'], axis=1)
train_x = train_x.dropna()
train_y = train_y.iloc[train_x.index]

In [8]:
model = ModelKeras(logging=True)
layer1 = Dense(1024, activation='relu', input_shape=(train_x.shape[1],))
layer2 = Dense(1024, activation='relu')
layer3 = Dense(1024, activation='relu')
layer3 = Dense(1024, activation='relu')
layer4 = Dense(1, activation='sigmoid')
model.add_layers(layer1, layer2, layer3, layer4)
model.compile()
model.set_fit_params(epochs=10, verbose=0)

[INFO] 2021-02-11 19:56:58,927 >>	=== NN KERAS MODEL ===


In [9]:
pred_train = get_pred_result(model, train_x, train_y)
get_acc_and_logloss(pred_train, train_y, logging=True)

[INFO] 2021-02-11 19:57:05,229 >>	ACCURACY: 0.6372549019607843
[INFO] 2021-02-11 19:57:05,230 >>	LOGLOSS: 0.6637444777466439
[INFO] 2021-02-11 19:57:05,231 >>	data size: 714
[INFO] 2021-02-11 19:57:05,232 >>	correct predictions: 455
[INFO] 2021-02-11 19:57:05,235 >>	


In [10]:
# Age standardized and target encoding
train_x, train_y, test_x = load_data()
base_path = '../features/nn/'
cols_to_rplc_train = {'Age': base_path + 'age_train_std_te.feather'}
train_x = load_feather(train_x, cols_to_rplc_train)

In [11]:
train_x = train_x.drop(['Name', 'Cabin', 'Ticket', 'Sex', 'Embarked'], axis=1)
train_x = train_x.dropna()
train_y = train_y.iloc[train_x.index]

In [17]:
pred_train = get_pred_result(model, train_x, train_y)
get_acc_and_logloss(pred_train, train_y, logging=False)

data size: 891
correct predictions: 641
accuracy: 0.7194163861
logloss: 0.5751301250


In [13]:
# Age standardized and mean
base_path = '../features/nn/'
cols_to_rplc_train = {'Age': base_path + 'age_train_std_mean.feather'}
train_x = load_feather(train_x, cols_to_rplc_train)

In [18]:
pred_train = get_pred_result(model, train_x, train_y)
get_acc_and_logloss(pred_train, train_y, logging=False)

data size: 891
correct predictions: 646
accuracy: 0.7250280584
logloss: 0.5590738576
